In [1]:
from qdrant_client import QdrantClient, models
from fastembed import TextEmbedding
import numpy as np

/Users/m/code/course-llm-zoomcamp/module02/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
client = QdrantClient("http://localhost:6333")

In [3]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"
query = 'I just discovered the course. Can I join now?'

In [ ]:
# Define the collection name
collection_name = "zoomcamp-hw02-q01"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

In [ ]:
points = []

point = models.PointStruct(
    id=0,
    vector=models.Document(text=query, model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
    payload={
        "text": query,
    } #save all needed metadata fields
)
points.append(point)

In [ ]:
points

In [ ]:
client.upsert(
    collection_name=collection_name,
    points=points
)

In [ ]:
# Get vectors from the collection
vectors = client.retrieve(
    collection_name=collection_name,
    ids=[0, 1],  # we'll retrieve the vector for point with id=0
    with_vectors=True  # include vectors in the response
)

In [ ]:
print(min(vectors[0].vector))

In [ ]:
q2 = 'Can I still join the course after the start date?'

In [ ]:
point = models.PointStruct(
    id=1,
    vector=models.Document(text=q2, model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
    payload={
        "text": q2,
    } #save all needed metadata fields
)
points.append(point)

client.upsert(
    collection_name=collection_name,
    points=points
)

In [ ]:
# Get vectors from the collection
vectors = client.retrieve(
    collection_name=collection_name,
    ids=[0, 1],  # we'll retrieve the vector for point with id=0
    with_vectors=True  # include vectors in the response
)

In [ ]:
q1_vector = vectors[0].vector
q2_vector = vectors[1].vector

In [ ]:
print(q1_vector)
print("\n\n\n")
print(q2_vector)

In [ ]:
# Compute cosine similarity between q1_vector and q2_vector
def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    return a.dot(b)

similarity = cosine_similarity(q1_vector, q2_vector)
print(f"Cosine similarity between vector 0 and vector 1: {similarity}")